## Procesos

A que llamamos proceso? Es como una abstraccion que tiene el SO de una tarea que se esta ejecutando.

### Que informacion tiene?

- El codigo del programa (ejecutable). 

- Datos que se generaron al programar. Cuando uno genera un programa y compila no solo se generan instrucciones sino que tambien bloques de datos. 

- Datos generados durante la ejecucion del programa. En la medida que el programa se va ejecutando el programa puede ir pidiendo memoria o ir modificando variables.

- Recursos virtuales (que archivos abrio, buffers de pantallas, semaforos, mutex, conexiones de red, etc).

- Informacon de prioridad de ejecucion y tiempo de ejecucion

- Informacion de seguridad (usuario duenio del proceso). Cuando el SO quiere acceder a un recurso va a verificar que el usuario duenio de ese programa tenga los permisos para esa accion.

- En el caso de que el proceso no este corriendo tengo que guardar toda la informacion de los registros como estaban.

Toda esa informacion el SO la tiene en un arreglo de estrucuturas (process table) donde mantiene la informacion de cada una de las tareas que se estan ejecutando. En particular todos los SO entregan algun numero y que puede identificar que proceso es, PID para linux por ejemplo. Ese ID proceso puede pensarse como el indice de esa tabla.

## Multiprocesamiento

La capacidad de tener multiples procesadores en una misma computadora es algo que es muy posterior a que se pusiera uno solo, durante muchisimas decadas habia un solo procesador en la mayoria de las maquinas. Sin embargo esto no quito que se haya implementado el multiprocesamiento hace mucho tiempo. Ejecutar mucho procesos a la vez es anterior a que exisitieran muchos procesadores. Como se hacia con un procesador y muchos procesos? el procesador normalmente es mucho mas rapido que los dispositivos de I/O, entonces durante muchisimo tiempo lo que se hacia era dediciar ese procesador un pequenio tiempo para cada tarea e ir saltando de una tarea a la otra. Se daba una apariencia de que todos los procesos corrian a la vez cuando en realidad sucedia de forma secuencial. Ese tiempo durante el cual ejecuta cada tarea se denomina timeslice, y escencialmente es del orden de los milisegundos. El tiempo se convierte en virtual, cuando tenemos un proceso con un procesador dedicado exclusivamente, ese proceso va a terminar la misma cantidad de clocks cada vez que lo ejecutas, porque en definitiva requiere una cierta cantidad de instrucciones y cada una de estas instrucciones tiene predefinida una cierta cantidad de clocks. Desde el momento en que vamos a compartrir el procesador ya no es exactamente lo mismo, si comparto con mas me toca menos y por ende mi tarea va a demorar mas. Entonces un tema fundamental del multiprocesamiento es que el tiempo se convierte en algo virtual. La realidad es que esto no es malo, como el procesador es mas rapido que todo, si dedicamos el procesador a una tarea pasaria muchsimo tiempo haciendo nada. El hecho de poder repartir el procesador en distitnas tareas es algo que beneficia el uso del procesador, se lo aprovecha mas.

## Vida de un proceso: Creacion

Los procesos en realidad nacen de alguna ejecucion, alguien corre un proceso. Algunos corren por el solo inicio del sistema, por ej el shell. Pero tambien pueden arrancar porque otro proceso lo dispara. En general como los procesos los mantiene el SO, lanzar un nuevo proceso implica llamar alguna funcion. En windows es CreateProcessAsUser() se le pasa un ejecutable con las instrucciones a ejecutar, variables de entorno (cjto de variables que el SO le provee a cada programa que arranca, tienen informacion variada). En windows tambien se pueden hacer algunos ajustes menores como SetPriorityClass(). En linux es un poco mas complejo, la unica formaa de crear un proceso es utilizando fork(). El SO agarra el proceso como esta y lo duplica, a partir de ahi hay dos procesos corriendo a la vez, un proceso devuelve 0 que es el proceso hijo y el proceso padre recibe el PID del hijo que se acaba de crear. Eso es incompleto, uno hace algunos ajustes menores como cambiar el usuario, y luego hay una funcion execve() que reemplaza el ejecutable del hijo por otro. Tenemos un proceso ejecutando que quiere lanzar a otro proceso, llamamos a fork y este duplica el proceso, el nuevo proceso sale del fork en el mismo programa que estaba. En android es mas facil, simplemente es una funcion. Lo que es importante entender cuando se lanza un nuevo proceso, es que ese nuevo proceso es un ente totalemnte independiente dentro del SO. A partir del fork, si el padre cambia cambia la suya, si el hijo cambia cambia la suya.

## Vida de un proceso: Terminacion

Otra vez tenemos varias alterantivas. Una es por voluntad propia, el proceso termino lo que tiene para hacer, o detecta condiciones tales en que ya no puede seguir adelante. Se le suele pasar al codigo un codigo de retorno se suele usar 0 si termino todo bien y negativo si hay algun error. Otra alternativa es que el SO es el que quiere terminar la tarea, por ej, abrimos un archivo, ves el programa, ves lo permisos, y ves que el programa no tiene permiso, por ende se cierra el programa. El programa tambien puede estar corrupto, el SO lo cierra. Ultima alternativa es que el proceso no quiere terminar pero algun usuario mata el proceso, por ej en el task manager. El SO se fijara que el usuario tenga el permiso y se cerrara el proceso si es que lo tiene. En los ultimos dos casos no se devuelve un codigo de resultado.

## Vida de un proceso: Estado

No es tan optimo que el procesador arbitrariramente asigne tiempo a los procesos. El SO asocia un estado a cada uno de los procesos. En windows tenes el estado ready to run que en cuanto tenga un timeslice se lo tengo que dar, running es lo que se estan ejecutando en ese momento. En linux es igual, cambian los nombres, pero es lo mismo. Suspended o Stopped son procesos que estan detenidos por un usuario. El Zombie o el ended, cuando un proceso lanza procesos hijos y llama al procesador diciendo que no tiene mas nada que hacer y salgo, el procesador no puede decir lo saco de memoria y listo porque el hijo puede requerir algo del padre. Entonces cuando se llama a exit process en vez de borrarlo del todo queda en ese estado ended o zombie. Luego estan block/waiting que en caso de linux esta dividio en dos interrupbtible sleep y uninterruptible sleep.

## Threads

Nosotros deciamos que los procesos hasta ahora eran algo lineal, un unico puntero indicando donde esta ejecutando. En realidad es bastante ineficiente cuando uno tiene que hacer varias tareas usar ese concepto. Por ejemplo quiero bajar de internet 10 archivos, podria bajar el primero, terminar empezar con el segundo, terminar, etc. La realidad es como lo lento es ese proceso de entrada y salida voy a perder mucho tiempo esperando. Es mucho mas eficiente partir en vez de un unico hilo de ejecucion, partir ese hilo en multiples hilos de ejecucion simultaneos que todos salen a hacer un pedido, todos esperan pero luego las respuestas vienen todas juntas. Aparece el concepto de hilos de ejecucion o threads, en el cual un proceso puede lanzar otros hilos de ejecucion y esos hilos cada uno hace una tarea independiente. Cada uno de estos hilos va a tener un puntero de ejecucion, un stack propio, sin embargo la diferencia principal entre proceso y threads es que hay una proteccion entre procesos mientras que en threads son todos hermanos, todos acceden a todos dentro de ese proceso mientras que no se salgan de ese proceso. Entonces lo unico que difiere entre procesos y threads es que en procesos hay un espacio de memoria unico y hay una proteccion entre procesos, un proceso no puede acceder a los recursos de otro. Los threads tambien son hilos de ejecucion que tienen su propio codigo pero estan metidos dentro de una bolsa que es el proceso, y no hay proteccion de memoria. Si vos creaste una variable en el main del programa, cualquier thread puede acceder a la variabel. Todos esos threads que son todos posibles hilos son los que despues el SO va a reparir en los procesadores para ir ejecutandolos, si puede ejecutar varios a la vez buenisimo, si tengo un solo procesador los ejecutare de a uno. Un proceso puede tener un thread o puede tener mas, si tiene uno solo, tiene un unico hilo de ejecucion ese proceso. Ya no nos alcanza con un process table, necesitamos partir eso y tener un punto de ejecucion por thread. Hay otras cosas que se mantienen en la tabla de procesos, por ejemplo los archivos. El SO va a distribuir la capacidad de ejecucion entre los threads que esten en estado ejecutable. Fijarse que el estado ahora es propio del thread, no solamente del proceso.

## Context switching: costo

El proceso es una entidad que cuando voy a sacar un proceso de la cpu y voy a poner otro, ese nuevo proceso no puede acceder a nada de lo que podia acceder el otro proceso, cuando voy a sacar un thread y poner otro, tengo menos cosas que hacer. Por ejemplo, cuando uno cambia de un thread a otro, como tengo menos cosas que modificar, el costo de pasarme de un thread a otro en unidades de tiempo es menor que para pasar de un proceso a otro. Los procesos por ej no comparten memoria, entonces una de las cosas que hacen cuando se cambia de proceso es borrar el cache, no tiene sentido conservar el cache cuando se cambio de procesos, el cache de codigo depende de si es el mismo programa o no el proceso al que corresponde. Lo mismo sucede con el TLB. Cambiar de proceso involucra entonces muchas mas cosas que cambiar de threads, por ende cambiar de threads desperdicia menos tiempo.

## Prioridades

Todos los SO manejan el concepto de prioridades, pueden ser numeros que se asignan arbitrariamente, y hay otros que son dinamicas las prioirdades. Por ej, cuando una tarea esta en el frente (interactuando con el usuario) entonces se le suele dar como un plus de velocidad. Si uno la manda atras ese plus lo pierde y lo recupera otra. Segundo, el deseo del usuario que puede haber subido la prioridad de un proceso para que termine antes. Tercero, acceso a ciertos dispositivos, hay ciertos dispositovs que son criticos, por ejemplo, la grabacion de un dvd, uno tiene que mandar strings de bits constantes desde que arranca hasta que termina, si ese string se interrumpe el disco se arruina. Por eso, estos procesos requieren una prioridad alta. Y en algunos casos, si es posible determinar cuanto le falta a la tarea, muchas veces es preferible darle prioridad y sacarsela de encima, sacandomela de encima libero, espacio, recursos, etc.

## Scheduling

Es el proceso de determinar a quien le voy a dar el siguiente timeslice. Soy el SO, tengo montones de threads, a cual le voy a dar el siguiente timeslice? La cosa no es trivial, podria decir que elija el que tenga mayor prioridad y le doy a ese pero de esa forma el de menor prioridad quedara trabada. Cuando uno tiene montones de recursos esto obviamente es menos importante, pero cuando hay una limitacion de recursos no. El hecho de recibir un timeslice no quiere decir que el proceso lo use en su totalidad, el SO no va a conservar ese resto de timeslice si el proceso no tiene nada que hacer. Tambien puede haber una interrupcion, en donde una tarea de muy alta prioridad puede inducir una suspension en una tarea de menor prioridad para seguir con la de alta. Otra cosa a tener en cuenta es que hay que ver a que proceso corresponde cada thread, hay que poner un limite por proceso. Hay otro concepto llamado priority inversion que se da cuando una tarea de alta prioirdad necesita un recurso que esta siendo accedido por uno de baja, no se le puede dar el recurso a la tarea de alta prioridad poruqe esta siendo utilizado por otro, lo que puedo hacer es invertir la prioridad para asi liberar el recurso y hacer funcionar la tarea.

Hay multiples algoritmos de scheduling. Una alternativa es el shortest-job-first: le doy la prioridad al trabajo que menos le falta para que libere los recursos. First-come-first-serve: le doy prioridad a las tareas por orden de llegada. Scheduling por prioridades, lo que hacen la mayor parte de los SO hoy, se le da mas timeslice a los que tienen mas prioridad que otras, si tienen la misma prioridad se utiliza alguno de los otros algoritmos. Highest response ratio next: trato de no cometer errores. Estos schedulings son por procesos. Dentro de ese proceso hay threads y hay que elegir a que thread le doy prioirdad. Round robin: orden, Multilevel feedback queuee: se crean colas por prioridad, greedy algorithm: consiste en tomar la mejor eleccion en cada paso esperando que eso lleve a la mejor solucion total eso no garantiza optimalidad.

## Collaborative Multiprocessing/Multithreading

El multiprocesameinto no estaba soportado por hardware, de hecho algo del DOS y bastante del windows 3.11, se podrian abrir varias ventanas pero corrian sobre un procesador que no soportaba eso.

## Preemptive Multiprocessing/Multithreading